In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px



from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler , LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from scipy.stats import mode


# from xgboost import XGBClassifier
# from catboost import CatBoostClassifier
# from lightgbm import LGBMClassifier

# import tensorflow as tf
# from tensorflow.keras import layers
# from tensorflow.keras import callbacks
# from tensorflow.keras.utils import to_categorical

from matplotlib import ticker
import time
import warnings
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('float_format', '{:f}'.format)
warnings.filterwarnings('ignore')

In [ ]:
train = pd.read_csv("../input/tabular-playground-series-dec-2021/train.csv")
test = pd.read_csv("../input/tabular-playground-series-dec-2021/test.csv")
submission = pd.read_csv("../input/tabular-playground-series-dec-2021/sample_submission.csv")


train.drop(["Id"] , axis = 1 , inplace = True)
test.drop(["Id"] , axis = 1 , inplace = True)
TARGET = 'Cover_Type'
FEATURES = [col for col in train.columns if col not in ['id', TARGET]]
RANDOM_STATE = 12 

In [ ]:
train = train.drop(index = int(np.where(train["Cover_Type"] == 5 )[0]))
train = train.drop(labels = ["Soil_Type7" , "Soil_Type15"] ,axis = 1)
FEATURES.remove('Soil_Type7')
FEATURES.remove('Soil_Type15')

In [ ]:
train.loc[train["Hillshade_9am"] < 0, "Hillshade_9am"] = 0
test.loc[test["Hillshade_9am"] < 0, "Hillshade_9am"] = 0

train.loc[train["Hillshade_Noon"] < 0, "Hillshade_Noon"] = 0
test.loc[test["Hillshade_Noon"] < 0, "Hillshade_Noon"] = 0

train.loc[train["Hillshade_3pm"] < 0, "Hillshade_3pm"] = 0
test.loc[test["Hillshade_3pm"] < 0, "Hillshade_3pm"] = 0

train.loc[train["Hillshade_9am"] > 255, "Hillshade_9am"] = 255
test.loc[test["Hillshade_9am"] > 255, "Hillshade_9am"] = 255

train.loc[train["Hillshade_Noon"] > 255, "Hillshade_Noon"] = 255
test.loc[test["Hillshade_Noon"] > 255, "Hillshade_Noon"] = 255

train.loc[train["Hillshade_3pm"] > 255, "Hillshade_3pm"] = 255
test.loc[test["Hillshade_3pm"] > 255, "Hillshade_3pm"] = 255

In [ ]:

df = pd.concat([train[FEATURES], test[FEATURES]], axis=0)

cat_features = [col for col in FEATURES if df[col].nunique() < 25]
cont_features = [col for col in FEATURES if df[col].nunique() >= 25]

del df



In [ ]:
scaler = StandardScaler()
train[cont_features] = scaler.fit_transform(train[cont_features])

In [ ]:
train.tail(10)

In [ ]:
X = train[FEATURES].to_numpy().astype(np.float32)
y = train[TARGET].to_numpy().astype(np.float32)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()

In [ ]:
rf.fit(X_train,y_train)

In [ ]:
pred = rf.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix
cf_m = confusion_matrix(y_test, pred)
cf_m

In [ ]:
for r in cf_m:
    print (sum(r))
    print (r/sum(r))

In [ ]:
# pd.read_csv(r'')
test_final = pd.read_csv("../input/tabular-playground-series-dec-2021/test.csv")

In [ ]:
test_final.loc[test_final["Hillshade_9am"] < 0, "Hillshade_9am"] = 0

test_final.loc[test_final["Hillshade_Noon"] < 0, "Hillshade_Noon"] = 0


test_final.loc[test_final["Hillshade_3pm"] < 0, "Hillshade_3pm"] = 0


test_final.loc[test_final["Hillshade_9am"] > 255, "Hillshade_9am"] = 255


test_final.loc[test_final["Hillshade_Noon"] > 255, "Hillshade_Noon"] = 255


test_final.loc[test_final["Hillshade_3pm"] > 255, "Hillshade_3pm"] = 255

In [ ]:
test_final[cont_features] = scaler.fit_transform(test_final[cont_features])

In [ ]:
test_final.drop('Soil_Type7',axis=1, inplace = True)
test_final.drop('Soil_Type15',axis=1, inplace = True)
# FEATURES.remove('Soil_Type15')

In [ ]:
test_final = test_final[FEATURES]

In [ ]:
pred_final = rf.predict(test_final)

In [ ]:
df = pd.DataFrame(pred_final, columns = ['Column_A'])

In [ ]:
test_final2 = pd.read_csv("../input/tabular-playground-series-dec-2021/test.csv")


In [ ]:

df['Id'] = test_final2['Id']

In [ ]:
df.tail()